# BERT(Bidirectional Encoder Representations from Transformers)

## 1. BERT의 개요

![](https://wikidocs.net/images/page/35594/%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.PNG)
<br />

BERT는 이전 챕터에서 배웠던 트랜스포머를 이용하여 구현되었으며, 위키피디아와 같이 레이블이 없는 텍스트 데이터로 훈련된 언어 모델입니다.
<br />

BERT가 높은 성능을 얻을수 있었던 것은, 레이블이 없는 방대한 데이터로 사전 훈련된 모델을 가지고, 레이블이 있는 다른 작업에서 추가 훈련과 함께 하이퍼파라미터를 재조정하여 이 모델을 사용하면 성능이 높게 나오는 기존의 사례들을 참고하였기 때문입니다. 다른 작업에 대해서 파라미터 재조정을 위한 추가 훈련 과정을 파인 튜닝(Fine-tunning)이라고 합니다.
<br />

위의 그림이 BERT의 파인 튜닝 사례입니다.

## 2. BERT의 크기

![](https://wikidocs.net/images/page/35594/bartbase%EC%99%80large.PNG)
<br />

BERT의 기본 구조는 트랜스포머의 인코더를 쌓아올린 구조입니다. 
<br />

## 3. BERT의 문맥을 반영한 임베딩(Contextual Embedding)

BERT는 Contextual Embedding을 사용하고 있습니다.
<br />

BERT의 입력은 Embedding Layer를 지난 임베딩 벡터들입니다. BERT의 연산을 거친 후의 출력 임베딩은 문장의 문맥을 모두 참고한 문맥을 반영한 임베딩이 됩니다. 

## 4. BERT의 서브워드 토크나이저 : WordPiece

BERT는 단어보다 더 작은 단위로 쪼개는 서브워드 토크나이저를 사용합니다. BERT가 사용한 토크나이저는 WordPiece 토크나이저로 서브워드 토크나이저 챕터에서 공부한 Byte Pair Encoding의 유사 알고리즘입니다. 동작 방식은 BPE와 조금 다르지만, 글자로부터 서브워드들을 병합해가는 방식으로 최종 단어 집합을 만드는 것은 BPE와 유사합니다.
<br />

서브워드 토크나이저는 기본적으로 자주 등장하는 단어는 그대로 단어 집합에 추가하지만, 자주 등장하지 않는 단어의 경우에는 더 작은 단위인 서브워드로 분리되어 서브워드들이 단어 집합에 추가된다는 아이디어를 갖고있습니다.
<br />

BERT에서 토큰화를 수행하는 방식은 다음과 같습니다.

> 준비물 : 이미 훈련 데이터로부터 만들어진 단어 집합
>
> 1. 토큰이 단어 집합에 존재한다.  
> => 해당 토큰을 분리하지 않는다.
>
> 2. 토큰이 단어 집합에 존재하지 않는다.    
> => 해당 토큰을 서브워드로 분리한다.  
> => 해당 토큰의 첫 번째 서브워드를 제외한 나머지 서브워드들은 앞에 '##'를 붙인 것을 토큰으로 한다.
<br />


## 5. Position Embedding

트랜스포머에서는 Positional Encoding이라는 방법을 통해서 단어의 위치 정보를 표현했습니다. Positional Encoding은 사인 함수와 코사인 함수를 사용하여 위치에 따라 다른 값을 가지는 행렬을 만들어 이를 단어 벡터들과 더하는 방법입니다.
<br />

BERT에서는 이와 유사하지만, 위치 정보를 사인 함수와 코사인 함수로 만드는 것이 아닌 학습을 통해서 얻는 Position Embedding이라는 방법을 사용합니다.
<br />

![](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC5.PNG)
<br />

위의 그림은 포지션 임베딩을 사용하는 방법을 보여줍니다. 우선, 위의 그림에서 WordPiece Embedding은 우리가 이미 알고있는 단어 임베딩으로 실질적인 입력입니다. 그리고 이 입력에 포지션 임베딩을 통해서 위치 정보를 더해주어야 합니다. 포지션 임베딩의 아이디어는 굉장히 간단한데, 위치 정보를 위한 임베딩 층을 하나 더 사용합니다.
<br />

실제 BERT에서는 문장의 최대 길이를 512로 하고 있으므로, 총 512개의 포지션 임베딩 벡터가 학습됩니다. 결론적으로 현재 설명한 내용을 기준으로는 BERT에서 총 두 개의 임베딩 층이 사용됩니다. 단어 집합의 크기가 30,522개인 단어 벡터를 위한 임베딩층과 문장의 최대 길이가 512이므로 512개의 포지션 벡터를 위한 임베딩 층입니다.


## 6. BERT의 사전 훈련(Pre-Training)

### 1) Masked Language Model, MLM

BERT는 사전 훈련을 위해서 인공 신경망의 입력으로 들어가는 입력 텍스트의 15%의 단어를 랜덤으로 마스킹(Masking)합니다. 그리고 인공 신경망에게 이 가려진 단어들을 예측하도록 합니다. 
<br />

더 정확히는 전부 [MASK]로 변경하지는 않고, 랜덤으로 선택된 15%의 단어들은 다시 다음과 같은 비율로 규칙이 적용됩니다.
<br />

- 80%의 단어들은 [MASK]로 변경

- 10% 단어들은 랜덤으로 다른 단어로 변경

- 10%의 단어들은 변경하지 않고 유지
<br />

이렇게 하는 이유는 [MASK]만 사용할 경우에는 [MASK] 토큰이 파인 튜닝 단계에서는 나타나지 않으므로 사전 학습 단계와 파인 튜닝 단계에서의 불일치가 발생하는 문제가 있습니다. 이 문제를 완화하기 위해 위와 같은 방법을 사용하고, 그림으로 나타내면 아래와 같습니다.
<br />

![](https://wikidocs.net/images/page/115055/%EC%A0%84%EC%B2%B4%EB%8B%A8%EC%96%B4.PNG)
<br />

'My dog is cute. he likes playing'이라는 문장에 대해서 Masked Language Model을 학습하고자 합니다. 해당 데이터셋이 아래와 같이 변경되었다고 가정하고 어떻게 학습하는지를 보겠습니다.
<br />

- 'dog' -> [MASK]

- 'he' -> 'king'

- 'play' -> 'play'
<br />

![](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC9.PNG)
<br />

BERT는 랜덤 단어 'king'으로 변경된 토큰에 대해서도 원래 단어가 무엇인지, 변경되지 않은 단어 'play'에 대해서도 원래 단어가 무엇인지를 예측해야 합니다. BERT는 이러한 Masked Language Model외에도 다음 문장 예측이라는 또 다른 태스크를 학습합니다.

### 2) Next Sentence Prediction, NSP

BERT는 두 개의 문장을 준 후에 이 문장이 이어지는 문장인지 아닌지를 맞추는 방식으로 훈련합니다. 이를 위해서 50:50 비율로 실제 이어지는 두 개의 문장과 랜덤으로 이어붙인 두 개의 문장을 주고 훈련시킵니다. 이를 각각 Sentence A와 Sentence B라고 하였을 때, 아래 예시는 문장의 연속성을 확인한 경우와 그렇지 않은 경우를 보여줍니다.
<br />

- 이어지는 문장의 경우

    - Sentence A : The man went to the store.

    - Sentence B : He bought a gallon of milk.

    - Label : IsNextSentence
<br />

- 이어지는 문장이 아닌 경우

    - Sentence A : The man went to the store.

    - Sentence B : dogs are so cute.

    - Label : NotNextSentence
<br />

![](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC10.PNG)
<br />

BERT의 입력으로 넣을 때에는 [SEP]라는 특별 토큰을 사용해서 문장을 구분합니다. 첫 번째 문장의 끝에 [SEP] 토큰을 넣고, 두 번째 문장이 끝나면 역시 [SEP] 토큰을 붙여줍니다. 그리고 이 두 문장이 실제 이어지는 문장인지 아닌지를 [CLS] 토큰의 위치의 출력층에서 이진 분류 문제를 풀도록 합니다. [CLS]토큰은 BERT가 분류 문제를 풀기 위해 추가된 특별 토큰입니다. 그리고 위의 그림에서 나타난 것과 같이 Masked Language Model과 Next Sentence Prediction은 따로 학습하는 것이 아닌 loss를 합하여 학습이 동시에 이루어집니다.
<br />

BERT가 이러한 NSP Task도 학습하는 이유는 Question Answering이나 Natural Language Inference와 같이 두 문장의 관계를 이해하는 것이 중요한 Task들도 해결하고자 하기때문입니다.

## 7. Segment Embedding

![](https://wikidocs.net/images/page/115055/%EA%B7%B8%EB%A6%BC7.PNG)
<br />

BERT에서는 문장 구분을 위해 Segment Embedding이라는 또 다른 임베딩 층을 사용합니다. 첫 번째 문장에는 Sentence 0 임베딩, 두 번째 문장에는 Sentence 1 임베딩을 더해주는 방식이며 임베딩 벡터는 두 개만 사용됩니다.
<br />

결론적으로 BERT는 총 3개의 임베딩 층이 사용됩니다.
<br />

- WordPiece Embedding : 실질적인 입력이 되는 워드 임베딩

- Position Embedding : 위치 정보를 학습하기 위한 임베딩

- Segment Embedding : 두 개의 문장을 구분하기 위한 임베딩

## 8. Fine-tuning BERT

### 1) Single Text Classification

![](https://wikidocs.net/images/page/115055/apply1.PNG)
<br />

입력된 문서에 대해서 분류를 하는 유형으로 문서의 시작에 [CLS]토큰을 입력합니다. 분류 문제를 풀기 위해서 [CLS]토큰 위치의 출력층에서 Dense layer를 추가하여 분류를 하게 됩니다.
<br />

### 2) Tagging

![](https://wikidocs.net/images/page/115055/apply2.PNG)
<br />

입력 텍스트의 각 토큰의 위치에 Dense layer를 사용하여 예측
<br />

### 3) Text Pair Classification or Regression

![](https://wikidocs.net/images/page/115055/apply3.PNG)
<br />

대표적으로 자연어 추론(Natural Language Inference)이 있습니다. 자연어 추론 문제란, 두 문장이 주어졌을 때, 하나의 문장이 다른 문장과 논리적으로 어떤 관계에 있는지를 분류하는 것입니다. 유형으로는 모순(contradiction), 함의(entailment), 중립(neutral)이 있습니다.
<br />

### 4) Question Answering

![](https://wikidocs.net/images/page/115055/apply4.PNG)
<br />

질문과 본문을 입력받으면, 본문의 일부분을 추출해서 질문에 답변하는 것입니다.

# BERT 구현

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K

import re
import math
import numpy as np
import pandas as pd
import random
import collections
import json
import shutil
import zipfile
import copy
from datetime import datetime

import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus']=False

import sentencepiece as spm
from tqdm.notebook import tqdm

In [16]:
# 몇 가지 유틸리티 함수 정의
def get_pad_mask(tokens, i_pad= 0):
    """
    pad mask 계산
    param tokens : tokens(bs, n_seq)
    param i_pad : id of pad
    return mask : pad mask (pad:1, other:0)
    """

    mask = tf.cast(tf.math.equal(tokens, i_pad), tf.float32)
    mask = tf.expand_dims(mask, axis=1)
    return mask

def get_ahead_mask(tokens, i_pad=0):
    n_seq = tf.shape(tokens)[1]
    ahead_mask = 1 - tf.linalg.band_part(tf.ones((n_seq, n_seq)), -1, 0)
    ahead_mask = tf.expand_dims(ahead_mask, axis=0)
    pad_mask = get_pad_mask(tokens, i_pad)
    mask = tf.maximum(ahead_mask, pad_mask)
    return mask

In [17]:
# GELU
@tf.function(experimental_relax_shapes=True)
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [18]:
def kernel_initializer(stddev=0.02):
    return tf.keras.initializers.TruncatedNormal(stddev=stddev)

def bias_initializer():
    return tf.zeros_initializer

In [19]:
class Config(dict):
    """
    json을 config 형태로 사용하기 위한 Class
    param dict : config dictionary
    """

    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        """
        file에서 Config를 생성함
        param file : filename
        """
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)
    

In [20]:
class SharedEmbedding(tf.keras.layers.Layer):
    def __init__(self, config, name='weight_shared_embedding'):
        super().__init__(name=name)
        self.n_vocab = config.n_vocab
        self.d_model = config.d_model
    
    def build(self, input_shape):
        with tf.name_scope("shared_embedding_weight"):
            self.shared_weights = self.add_weight('weights', shape=[self.n_vocab, self.d_model], initializer=kernel_initializer())
    
    def call(self, inputs, mode='embedding'):
        if mode == 'embedding':
            return self._embedding(inputs)
        elif mode == 'linear':
            return self._linear(inputs)
        else:
            raise ValueError(f"mode {mode} is not valid")
    
    def _embedding(self, inputs):
        embed = tf.gather(self.shared_weights, tf.cast(inputs, tf.int32))
        return embed
    
    def _linear(self, inputs):
        n_batch = tf.shape(inputs)[0]
        n_seq = tf.shape(inputs)[1]

        inputs = tf.reshape(inputs, [-1, self.d_model]) # (batch_size * n_seq, d_model)

        outputs = tf.matmul(inputs, self.shared_weights, transpose_b=True)
        outputs = tf.reshape(outputs, [n_batch, n_seq, self.n_vocab])
        return outputs

In [21]:
class PositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, config, name='position_embedding'):
        super().__init__(name=name)
        self.embedding = tf.keras.layers.Embedding(config.n_seq, config.d_model, embeddings_initializer=kernel_initializer())

    def call(self, inputs):
        position = tf.cast(tf.math.cumsum(tf.ones_like(inputs), axis=1, exclusive=True), tf.int32)
        embed = self.embedding(position)
        return embed

In [22]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self, name='scaled_dot_product_attention'):
        super().__init__(name=name)

    def call(self, Q, K, V, attn_mask):
        attn_score = tf.matmul(Q, K, transpose_b=True)
        scale = tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32))
        attn_scale = tf.math.divide(attn_score, scale)
        attn_scale -= 1e9 * attn_mask
        attn_prob = tf.nn.softmax(attn_scale, axis=-1)
        attn_out = tf.matmul(attn_prob, V)
        return attn_out

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, config, name='multi_head_attention'):

        super().__init__(name=name)

        self.d_model = config.d_model
        self.n_head = config.n_head
        self.d_head = config.d_head

        self.wq = tf.keras.layers.Dense(config.n_head * config.d_head, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
        self.wk = tf.keras.layers.Dense(config.n_head * config.d_head, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
        self.wv = tf.keras.layers.Dense(config.n_head * config.d_head, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())

        self.attention = ScaledDotProductAttention(name='self_attention')

        self.wo = tf.keras.layers.Dense(config.n_head , kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
    
    def call(self, Q, K, V, attn_mask):

        batch_size = tf.shape(Q)[0]
        qm = tf.transpose(tf.reshape(self.wq(Q), [batch_size, -1, self.n_head, self.d_head]), [0,2,1,3])
        km = tf.transpose(tf.reshape(self.wk(K), [batch_size, -1, self.n_head, self.d_head]), [0,2,1,3])
        vm = tf.transpose(tf.reshape(self.wv(V), [batch_size, -1, self.n_head, self.d_head]), [0,2,1,3])
        attn_mask_m = tf.expand_dims(attn_mask, axis=1)

        attn_out = self.attention(qm, km, vm, attn_mask_m)

        attn_out_m = tf.transpose(attn_out, perm=[0,2,1,3])
        attn_out = tf.reshape(attn_out_m, [batch_size, -1, self.n_head * self.d_head])
        attn_out = self.wo(attn_out)

        return attn_out

In [24]:
class PositionWiseFeedForward(tf.keras.layers.Layer):

    def __init__(self, config, name='feed_forward'):

        super().__init__(name=name)
        self.w1 = tf.keras.layers.Dense(config.d_ff, activation=gelu, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
        self.w2 = tf.keras.layers.Dense(config.d_model, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
    
    def call(self, inputs):
        ff_val = self.w2(self.w1(inputs))
        return ff_val

In [25]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, config, name='encoder_layer'):
        super().__init__(name=name)
        self.self_attention = MultiHeadAttention(config)
        self.norm1 = tf.keras.layers.LayerNormalization(epsilon=config.layernorm_epsilon)
        
        self.ffn = PositionWiseFeedForward(config)
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon=config.layernorm_epsilon)

        self.dropout = tf.keras.layers.Dropout(config.dropout)

    def call(self, enc_embed, self_mask):
        self_attn_val = self.self_attention(enc_embed, enc_embed, enc_embed, self_mask)
        norm1_val = self.norm1(enc_embed + self.dropout(self_attn_val))

        ffn_val = self.ffn(norm1_val)
        enc_out = self.norm2(norm1_val + self.dropout(ffn_val))

        return enc_out

In [26]:
class BERT(tf.keras.layers.Layer):
    def __init__(self, config, name='bert'):

        super().__init__(name=name)
        self.i_pad = config.i_pad
        self.embedding = SharedEmbedding(config)
        self.position = PositionEmbedding(config)
        self.segment = tf.keras.layers.Embedding(2, config.d_model, embeddings_initializer=kernel_initializer())
        self.norm = tf.keras.layers.LayerNormalization(epsilon=config.layernorm_epsilon)

        self.encoder_layers = [EncoderLayer(config, name=f"encoder_layer_{i}") for i in range(config.n_layer)]
        self.dropout = tf.keras.layers.Dropout(config.dropout)

    def call(self, inputs):
        enc_tokens, segments = inputs

        enc_self_mask = tf.keras.layers.Lambda(get_pad_mask, output_shape=(1, None), name='enc_self_mask')(enc_tokens, self.i_pad)

        enc_embed = self.get_embedding(enc_tokens, segments)

        enc_out = self.dropout(enc_embed)
        for encoder_layer in self.encoder_layers:
            enc_out = encoder_layer(enc_out, enc_self_mask)

        logits_cls = enc_out[:, 0]
        logits_lm = self.embedding(enc_out, mode='linear')
        return logits_cls, logits_lm
    
    def get_embedding(self, tokens, segments):
        embed = self.embedding(tokens) + self.position(tokens) + self.segment(segments)
        embed = self.norm(embed)
        return embed

In [27]:
class PooledOutput(tf.keras.layers.Layer):
    def __init__(self, config, n_output, name="pooled_output"):
        super().__init__(name=name)

        self.dense1 = tf.keras.layers.Dense(config.d_model, activation=tf.nn.tanh, kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
        self.dense2 = tf.keras.layers.Dense(n_output, use_bias=False, activation=tf.nn.softmax, name="nsp", kernel_initializer=kernel_initializer(), bias_initializer=bias_initializer())
 
    def call(self, inputs):
        outputs = self.dense1(inputs)
        outputs = self.dense2(outputs)
        return outputs

In [28]:
def build_model_pre_train(config):
    enc_tokens = tf.keras.layers.Input((None,), name="enc_tokens")
    segments = tf.keras.layers.Input((None,), name="segments")

    bert = BERT(config)

    logits_cls, logits_lm = bert((enc_tokens, segments))

    # [CLS]토큰의 마지막 hidden state를 이용해서 IsNext 인지 NotNext인지를 판단함.
    logits_cls = PooledOutput(config, 2, name="pooled_nsp")(logits_cls)
    outputs_nsp = tf.keras.layers.Softmax(name="nsp")(logits_cls)

    # masked token 예측
    outputs_mlm = tf.keras.layers.Softmax(name="mlm")(logits_lm)

    model = tf.keras.Model(inputs=(enc_tokens, segments), outputs=(outputs_nsp, outputs_mlm))
    return model
